In [44]:
import pandas as pd

In [45]:
# Load dataset
try:
    orders = pd.read_csv('./orders_dataset.csv')
    order_items = pd.read_csv('./order_items_dataset.csv')
    customers = pd.read_csv('./customers_dataset.csv')
    products = pd.read_csv('./products_dataset.csv')
    reviews = pd.read_csv('./order_reviews_dataset.csv')
except FileNotFoundError as e:
    print(f"Error: {e}. Pastikan semua file dataset tersedia.")
    raise

In [46]:
# Periksa kolom customer_id di masing-masing dataset
print("Kolom di orders:", orders.columns)
print("Kolom di order_items:", order_items.columns)
print("Kolom di products:", products.columns)
print("Kolom di customers:", customers.columns)
print("Kolom di reviews:", reviews.columns)

Kolom di orders: Index(['order_id', 'customer_id', 'order_status', 'order_purchase_timestamp',
       'order_approved_at', 'order_delivered_carrier_date',
       'order_delivered_customer_date', 'order_estimated_delivery_date'],
      dtype='object')
Kolom di order_items: Index(['order_id', 'order_item_id', 'product_id', 'seller_id',
       'shipping_limit_date', 'price', 'freight_value'],
      dtype='object')
Kolom di products: Index(['product_id', 'product_category_name', 'product_name_lenght',
       'product_description_lenght', 'product_photos_qty', 'product_weight_g',
       'product_length_cm', 'product_height_cm', 'product_width_cm'],
      dtype='object')
Kolom di customers: Index(['customer_id', 'customer_unique_id', 'customer_zip_code_prefix',
       'customer_city', 'customer_state'],
      dtype='object')
Kolom di reviews: Index(['review_id', 'order_id', 'review_score', 'review_comment_title',
       'review_comment_message', 'review_creation_date',
       'review_answer_

In [47]:
# Data Wrangling
# Mengubah tipe data waktu untuk analisis waktu
orders['order_purchase_timestamp'] = pd.to_datetime(orders['order_purchase_timestamp'])
orders['order_delivered_customer_date'] = pd.to_datetime(orders['order_delivered_customer_date'])

# Menambahkan kolom waktu pengiriman
orders['delivery_time'] = (orders['order_delivered_customer_date'] - orders['order_purchase_timestamp']).dt.days
orders['order_year'] = orders['order_purchase_timestamp'].dt.year

In [48]:
# Periksa apakah ada customer_id yang hilang (NaN)
print("Jumlah NaN di customer_id di orders:", orders['customer_id'].isna().sum())
print("Jumlah NaN di customer_id di customers:", customers['customer_id'].isna().sum())

Jumlah NaN di customer_id di orders: 0
Jumlah NaN di customer_id di customers: 0


In [49]:
# Penanganan missing value pada kolom waktu
orders['order_approved_at'] = orders['order_approved_at'].fillna(pd.Timestamp('1970-01-01'))  # Isi dengan nilai default
orders['order_delivered_customer_date'] = orders['order_delivered_customer_date'].fillna(orders['order_purchase_timestamp'])  # Isi dengan tanggal pembelian

In [50]:
# Gabungkan dataset untuk membuat main_data
main_data = pd.merge(order_items, orders[['order_id', 'order_purchase_timestamp', 'delivery_time', 'order_year', 'customer_id']], on='order_id', how='left')
main_data = pd.merge(main_data, products[['product_id', 'product_category_name']], on='product_id', how='left')
main_data = pd.merge(main_data, customers[['customer_id', 'customer_state']], on='customer_id', how='left')
main_data = pd.merge(main_data, reviews[['order_id', 'review_score']], on='order_id', how='left')

# Cek apakah ada nilai hilang pada 'price' dan kolom lainnya
print(main_data.isna().sum())

order_id                       0
order_item_id                  0
product_id                     0
seller_id                      0
shipping_limit_date            0
price                          0
freight_value                  0
order_purchase_timestamp       0
delivery_time               2475
order_year                     0
customer_id                    0
product_category_name       1612
customer_state                 0
review_score                 942
dtype: int64


In [51]:
# Pastikan tidak ada nilai hilang yang tidak diinginkan
main_data = main_data.dropna(subset=['order_purchase_timestamp', 'price'])  # Hanya menghapus data dengan NaN pada kolom 'order_purchase_timestamp' dan 'price'

In [52]:
# Periksa dataset akhir
print(main_data.info())
print(main_data.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 113314 entries, 0 to 113313
Data columns (total 14 columns):
 #   Column                    Non-Null Count   Dtype         
---  ------                    --------------   -----         
 0   order_id                  113314 non-null  object        
 1   order_item_id             113314 non-null  int64         
 2   product_id                113314 non-null  object        
 3   seller_id                 113314 non-null  object        
 4   shipping_limit_date       113314 non-null  object        
 5   price                     113314 non-null  float64       
 6   freight_value             113314 non-null  float64       
 7   order_purchase_timestamp  113314 non-null  datetime64[ns]
 8   delivery_time             110839 non-null  float64       
 9   order_year                113314 non-null  int32         
 10  customer_id               113314 non-null  object        
 11  product_category_name     111702 non-null  object        
 12  cu

In [53]:
# RFM Analysis
latest_purchase_date = orders['order_purchase_timestamp'].max()
recency = orders.groupby('customer_id')['order_purchase_timestamp'].max().reset_index()
recency['recency'] = (latest_purchase_date - recency['order_purchase_timestamp']).dt.days

frequency = orders.groupby('customer_id')['order_id'].nunique().reset_index()
frequency.columns = ['customer_id', 'frequency']

# Pastikan 'price' ada
monetary = main_data.groupby('customer_id')['price'].sum().reset_index()
monetary.columns = ['customer_id', 'monetary']

rfm = pd.merge(recency[['customer_id', 'recency']], frequency, on='customer_id', how='left')
rfm = pd.merge(rfm, monetary, on='customer_id', how='left')

In [54]:
# Gabungkan ke main_data
main_data = pd.merge(main_data, rfm, on='customer_id', how='left')

In [55]:
# Simpan file main_data
main_data.to_csv('main_data.csv', index=False)
print("File main_data.csv berhasil dibuat!")

File main_data.csv berhasil dibuat!


In [56]:
main_data.head(10)

,order_id,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value,order_purchase_timestamp,delivery_time,order_year,customer_id,product_category_name,customer_state,review_score,recency,frequency,monetary
0,00010242fe8c5a6d1ba2dd792cb16214,1,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,2017-09-19 09:45:35,58.90,13.29,2017-09-13 08:59:02,7.0,2017,3ce436f183e68e07877b285a838db11a,cool_stuff,RJ,5.0,399,1,58.90
1,00018f77f2f0320c557190d7a144bdd3,1,e5f2d52b802189ee658865ca93d83a8f,dd7ddc04e1b6c2c614352b383efe2d36,2017-05-03 11:05:13,239.90,19.93,2017-04-26 10:53:06,16.0,2017,f6dd3ec061db4e3987629fe6b26e5cce,pet_shop,SP,4.0,539,1,239.90
2,000229ec398224ef6ca0657da4fc703e,1,c777355d18b72b67abbeef9df44fd0fd,5b51032eddd242adc84c38acab88f23d,2018-01-18 14:48:30,199.00,17.87,2018-01-14 14:33:31,7.0,2018,6489ae5e4333f3693df5ad4372dab6d3,moveis_decoracao,MG,5.0,276,1,199.00
3,00024acbcdf0a6daa1e931b038114c75,1,7634da152a4610f1595efa32f14722fc,9d7a1d34a5052409006425275ba1c2b4,2018-08-15 10:10:18,12.99,12.79,2018-08-08 10:00:35,6.0,2018,d4eb9395c8c0431ee92fce09860c5a06,perfumaria,SP,4.0,70,1,12.99
4,00042b26cf59d7ce69dfabb4e55b4fd9,1,ac6c3623068f30de03045865e4e10089,df560393f3a51e74553ab94004ba5c87,2017-02-13 13:57:51,199.90,18.14,2017-02-04 13:57:51,25.0,2017,58dbd0b2d70206bf40e62cd34e84d795,ferramentas_jardim,SP,5.0,620,1,199.90
5,00048cc3ae777c65dbb7d2a0634bc1ea,1,ef92defde845ab8450f9d70c526ef70f,6426d21aca402a131fc0a5d0960a3c90,2017-05-23 03:55:27,21.90,12.69,2017-05-15 21:42:34,6.0,2017,816cbea969fe5b689b39cfc97a506742,utilidades_domesticas,MG,4.0,519,1,21.90
6,00054e8431b9d7675808bcb819fb4a32,1,8d4f2bb7e93e6710a28f34fa83ee7d28,7040e82f899a04d1b434b795a43b4617,2017-12-14 12:10:31,19.90,11.85,2017-12-10 11:53:48,8.0,2017,32e2e6ab09e778d99bf2e0ecd4898718,telefonia,SP,4.0,311,1,19.90
7,000576fe39319847cbb9d288c5617fa6,1,557d850972a7d6f792fd18ae1400d9b6,5996cddab893a4652a15592fb58ab8db,2018-07-10 12:30:45,810.00,70.75,2018-07-04 12:08:27,5.0,2018,9ed5e522dd9dd85b4af4a077526d8117,ferramentas_jardim,SP,5.0,105,1,810.00
8,0005a1a1728c9d785b8e2b08b904576c,1,310ae3c140ff94b03219ad0adc3c778f,a416b6a846a11724393025641d4edd5e,2018-03-26 18:31:29,145.95,11.65,2018-03-19 18:40:33,9.0,2018,16150771dfd4776261284213b89c304e,beleza_saude,SP,1.0,211,1,145.95
9,0005f50442cb953dcd1d21e1fb923495,1,4535b0e1091c278dfd193e5a1d63b39f,ba143b05f0110f0dc71ad71b4466ce92,2018-07-06 14:10:56,53.99,11.40,2018-07-02 13:59:39,2.0,2018,351d3cb2cee3c7fd0af6616c82df21d3,livros_tecnicos,SP,4.0,107,1,53.99
